In [25]:
import os

def load_data(filepath):
    """Charge un fichier et retourne une liste de blocs (groupes de 11 lignes)."""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Séparer les blocs de 11 lignes
        blocks = []
        current_block = []

        for line in lines:
            stripped_line = line.strip()

            # Ajouter la ligne au bloc actuel si elle n'est pas vide
            if stripped_line:
                current_block.append(stripped_line)

            # Lorsque le bloc atteint 11 lignes, on l'ajoute et on réinitialise
            if len(current_block) == 11:
                blocks.append("\n".join(current_block))
                current_block = []

        return blocks

    except FileNotFoundError:
        print(f"Erreur : Le fichier {filepath} est introuvable.")
        return []
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {filepath} : {e}")
        return []


def classify_and_save_blocks(blocks, classification, train_file, test_file):
    """
    Classifie les blocs en train et test, ajoute la classification et les sauvegarde.
    """
    for block in blocks:
        # Extraire la première ligne du bloc
        first_line = block.split("\n")[0].strip()

        # Vérifier le type de bloc
        if first_line.startswith('*'):  # Train
            file = train_file
        else:  # Test
            file = test_file

        # Ajouter le bloc et la classification au fichier
        file.write(block + "\n\n")
        file.write(str(classification) + "\n\n")


def save_datasets(mine_file, rocks_file, train_filepath, test_filepath):
    """Gère la séparation des blocs et leur classification."""
    try:
        # Charger les fichiers
        mine_blocks = load_data(mine_file)
        rocks_blocks = load_data(rocks_file)

        # Ouvrir les fichiers de sortie
        os.makedirs(os.path.dirname(train_filepath), exist_ok=True)
        with open(train_filepath, 'w', encoding='utf-8') as train_file, \
             open(test_filepath, 'w', encoding='utf-8') as test_file:

            # Classifier et sauvegarder les blocs des fichiers "mines" et "rocks"
            classify_and_save_blocks(mine_blocks, 1, train_file, test_file)  # Mines : classification = 1
            classify_and_save_blocks(rocks_blocks, -1, train_file, test_file)  # Rocks : classification = -1

    except Exception as e:
        print(f"Erreur lors de la création des datasets : {e}")


# Chemins vers les fichiers sources
mine_file = '../sonar_data/sonar.mines'
rocks_file = '../sonar_data/sonar.rocks'

# Chemins des fichiers de sortie
train_filepath = 'tp2/result_sets/train_set.txt'
test_filepath = 'tp2/result_sets/test_set.txt'

# Générer les datasets
save_datasets(mine_file, rocks_file, train_filepath, test_filepath)

print("Les fichiers train_set.txt et test_set.txt ont été créés avec succès !")


Les fichiers train_set.txt et test_set.txt ont été créés avec succès !


In [26]:
from classes.ensemble import Ensemble
from classes.paire import Paire


def lire_blocs(filepath):
    """
    Lit un fichier et retourne une liste de blocs (groupes de 14 lignes).

    Args:
        filepath (str): Le chemin vers le fichier à lire.

    Returns:
        list: Une liste de blocs, chaque bloc est une liste de lignes de vecteurs.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        blocs = []
        i = 0

        while i < len(lines):
            # Lire 14 lignes pour un bloc
            bloc = lines[i:i + 14]
            if len(bloc) < 14:
                break  # Sortir si le bloc n'est pas complet

            blocs.append(bloc)
            i += 14  # Passer au bloc suivant

        return blocs

    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {filepath} : {e}")
        return []


def traiter_bloc_en_paire(bloc):
    """
    Transforme un bloc en une paire de vecteurs et de classification.

    Args:
        bloc (list): Un bloc contenant 14 lignes (vecteurs, nom, classification).

    Returns:
        Paire: Une instance de `Paire` avec le vecteur et la classification.
    """
    vecteur = []

    # Lire les lignes de vecteurs (2ème à 11ème)
    for i in range(1, 12):  # On commence à la 1ère ligne (1) jusqu'à la 11ème
        ligne_donnees = bloc[i].strip('{} \n')
        if ligne_donnees:  # Ignore les lignes vides
            valeurs = [float(val) for val in ligne_donnees.split()]
            vecteur.extend(valeurs)

    # La 13ème ligne contient la classification
    classification_str = bloc[12].strip()  # La ligne vide avant la classification
    classification = 1 if classification_str == '1' else -1

    return Paire(tuple(vecteur), classification)


def construire_ensemble(filepath):
    """
    Crée un Ensemble à partir d'un fichier train ou test.

    Args:
        filepath (str): Le chemin vers le fichier à lire.

    Returns:
        Ensemble: Un objet `Ensemble` contenant toutes les paires lues depuis le fichier.
    """
    ensemble = Ensemble()
    try:
        blocs = lire_blocs(filepath)
        if not blocs:
            return ensemble

        for bloc in blocs:
            paire = traiter_bloc_en_paire(bloc)
            if paire:
                ensemble.ajouter_paire(paire)

    except Exception as e:
        print(f"Erreur lors de la création de l'ensemble depuis {filepath} : {e}")

    return ensemble


# Exemple d'utilisation
train_ensemble = construire_ensemble('tp2/result_sets/train_set.txt')
test_ensemble = construire_ensemble('tp2/result_sets/test_set.txt')

# Affichage pour validation
print("Train Ensemble contient :", len(train_ensemble.elements), "paires.")
print("Test Ensemble contient :", len(test_ensemble.elements), "paires.")
# print(train_ensemble)
# print(test_ensemble)

Train Ensemble contient : 104 paires.
Test Ensemble contient : 104 paires.


In [ ]:
def creer_all_ensemble(train_filepath, test_filepath):
    train_ensemble = construire_ensemble(train_filepath)
    test_ensemble = construire_ensemble(test_filepath)

    # Créer un nouvel ensemble qui combine les paires des deux ensembles
    all_ensemble = Ensemble(*train_ensemble.elements, *test_ensemble.elements)

    return all_ensemble


# Exemple d'utilisation
all_ensemble = creer_all_ensemble('tp2/result_sets/train_set.txt', 'tp2/result_sets/test_set.txt')

# Affichage pour validation
print("All Ensemble contient :", len(all_ensemble.elements), "paires.")
print(all_ensemble)